In [10]:
import pandas as pd
import numpy as np

## import the data

In [11]:
train = pd.read_csv('/Users/asaf/Google Drive/Work/Python Scripts/titanic/train.csv')


## Feature Selection

In [12]:
drop_elements = ['PassengerId', 'Name', 'Ticket','Parch','Cabin']
train = train.drop(drop_elements,axis = 1)

# Missing Values

In [13]:
train.isnull().mean() * 100

Survived     0.000000
Pclass       0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Fare         0.000000
Embarked     0.224467
dtype: float64

## Multiple Imputations by Chained Equations(MICE)

In [14]:
string_cols_full = list(train.loc[:, train.dtypes == np.object].columns[~train.loc[:, train.dtypes == np.object].isnull().any()])
string_cols_missing_v = list(train.loc[:, train.dtypes == np.object].columns[train.loc[:, train.dtypes == np.object].isnull().any()])

#encoding the columns without any missing values
train = pd.get_dummies(train,columns = string_cols_full, drop_first=True)

In [15]:
#from sklearn.impute import SimpleImputer
#imp = SimpleImputer(strategy="most_frequent")
#train['Cabin'] = imp.fit_transform(pd.DataFrame(train['Cabin']))

from fancyimpute import KNN
from sklearn.preprocessing import OrdinalEncoder

#instantiate both packages to use
encoder = OrdinalEncoder()
imputer = KNN()

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in string_cols_missing_v:
    encode(train[columns])


In [16]:
from fancyimpute import IterativeImputer
MICE_imputer = IterativeImputer()
train.iloc[:, :] = MICE_imputer.fit_transform(train)

In [17]:
train.isnull().mean() * 100

Survived    0.0
Pclass      0.0
Age         0.0
SibSp       0.0
Fare        0.0
Embarked    0.0
Sex_male    0.0
dtype: float64